# Trading

The purpose of this worksheet is to demonstrate and describe how trading works in OSEM.

In [ ]:
import os
import pandas as pd
from ImportData import get_configuration, get_settings, import_SWEiopa, get_Cash, get_EquityShare,get_corporate_bonds, get_Liability
from ConfigurationClass import Configuration
from CurvesClass import Curves
from BondClasses import CorpBondPortfolio
from EquityClasses import EquitySharePortfolio
from MainLoop import create_cashflow_dataframe, create_liabilities_df, set_dates_of_interest, process_expired_cf, process_expired_liab, trade

In [209]:
base_folder = os.getcwd()  # Get current working directory
conf: Configuration
conf = get_configuration(os.path.join(base_folder, "ALM.ini"), os)

In [210]:
parameters_file = conf.input_parameters
cash_portfolio_file = conf.input_cash_portfolio
equity_portfolio_file = conf.input_equity_portfolio
liability_cashflow_file = conf.input_liability_cashflow
bond_portfolio_file = conf.input_bond_portfolio

In [211]:
settings = get_settings(parameters_file)

In [212]:
[maturities_country, curve_country, extra_param, Qb] =  import_SWEiopa(settings.EIOPA_param_file,
                                                                          settings.EIOPA_curves_file, settings.country)


Curves object with information about term structure

In [214]:
curves = Curves(extra_param["UFR"] / 100, settings.precision, settings.tau, settings.modelling_date,
                    settings.country)

In [215]:
curves.SetObservedTermStructure(maturity_vec=curve_country.index.tolist(), yield_vec=curve_country.values)
curves.SetObservedTermStructure(maturity_vec=curve_country.index.tolist(), yield_vec=curve_country.values)
curves.CalcFwdRates()
curves.ProjectForwardRate(settings.n_proj_years+1)
curves.CalibrateProjected(settings.n_proj_years+1, 0.05, 0.5, 1000)  

Cash

In [217]:
cash = get_Cash(cash_portfolio_file)

In [218]:
equity_input_generator = get_EquityShare(equity_portfolio_file) # Create generator that contains all equity positions
eq_input = {equity_share.asset_id: equity_share for equity_share in equity_input_generator}

In [219]:
bond_input_generator = get_corporate_bonds(bond_portfolio_file)
bond_input = {corp_bond.asset_id: corp_bond for corp_bond in bond_input_generator}

In [220]:
eq_ptf = EquitySharePortfolio(eq_input)
bd_ptf = CorpBondPortfolio(bond_input)

In [221]:
div_dict = eq_ptf.create_dividend_flows(settings.modelling_date, settings.end_date)
ter_dict = eq_ptf.create_terminal_flows(modelling_date=settings.modelling_date,
                                                            terminal_date=settings.end_date,
                                                            terminal_rate=curves.ufr)
cpn_flows = bd_ptf.create_coupon_flows(settings.modelling_date, settings.end_date)
not_flows = bd_ptf.create_maturity_flows(terminal_date=settings.end_date)

In [222]:
unique_div_dates = eq_ptf.unique_dates_profile(div_dict)
unique_ter_dates = eq_ptf.unique_dates_profile(ter_dict)
unique_cpn_dates = bd_ptf.unique_dates_profile(cpn_flows)
unique_not_dates = bd_ptf.unique_dates_profile(not_flows)

In [223]:
liabilities = get_Liability(liability_cashflow_file)
unique_liabilities_dates = liabilities.unique_dates_profile()


In [224]:
[eq_price_df, eq_growth_df, eq_units_df] = eq_ptf.init_equity_portfolio_to_dataframe(settings.modelling_date)
[bd_price_df, bd_zspread_df, bd_units_df] = bd_ptf.init_bond_portfolio_to_dataframe(settings.modelling_date)
bank_account = pd.DataFrame(data=[cash.bank_account], columns=[settings.modelling_date])

In [225]:
div_df = create_cashflow_dataframe(div_dict, unique_div_dates)
ter_df = create_cashflow_dataframe(ter_dict, unique_ter_dates)
cpn_df = create_cashflow_dataframe(cpn_flows, unique_cpn_dates)
not_df = create_cashflow_dataframe(not_flows, unique_not_dates)
liab_df = create_liabilities_df(liabilities)

In [226]:
prev_mkt_value = sum(eq_price_df[settings.modelling_date] * eq_units_df[settings.modelling_date])+ sum(bd_price_df[settings.modelling_date] * bd_units_df[settings.modelling_date])  # Value of the initial portfolio

In [227]:
dates_of_interest = set_dates_of_interest(settings.modelling_date, settings.end_date)

In [228]:
previous_date = settings.modelling_date

In [229]:
proj_period = 0

In [230]:
bd_zspread_df=bd_ptf.calibrate_bond_portfolio(bd_zspread_df, settings, proj_period, curves)

In [231]:
for current_date in dates_of_interest.values:
    init_mkt_value = sum(eq_price_df[previous_date] * eq_units_df[previous_date]) + sum(bd_price_df[previous_date] * bd_units_df[previous_date])
    eq_units_df[current_date] = eq_units_df[previous_date]
    bd_units_df[current_date] = bd_units_df[previous_date]
    
    bank_account[current_date] = bank_account[previous_date]
        
    time_frac = (current_date - previous_date).days / 365.5

    cash, div_df, unique_div_dates = process_expired_cf(unique_div_dates, current_date, div_df, eq_units_df)
    bank_account[current_date] += cash

    cash, cpn_df, unique_cpn_dates = process_expired_cf(unique_cpn_dates, current_date, cpn_df, bd_units_df)
    bank_account[current_date] += cash

    cash, ter_df, unique_ter_dates = process_expired_cf(unique_ter_dates, current_date, ter_df, eq_units_df)
    bank_account[current_date] += cash

    cash, not_df, unique_not_dates = process_expired_cf(unique_not_dates, current_date, not_df, bd_units_df)
    bank_account[current_date] += cash

    cash, liab_df, unique_liabilities_dates = process_expired_liab(unique_liabilities_dates, current_date, liab_df)
        
    bank_account[current_date] -= cash

    eq_price_df[current_date] = eq_price_df[previous_date] * (
            1 + eq_growth_df[settings.modelling_date]) ** time_frac

    bd_price_df[current_date] = bd_price_df[previous_date]
        
    bd_price_df = bd_ptf.price_bond_portfolio(cpn_df, not_df, settings, proj_period, curves, bd_zspread_df, bd_price_df, current_date)
        
    total_market_value = sum(eq_units_df[current_date]*eq_price_df[current_date]) + sum(bd_price_df[current_date] * bd_units_df[current_date]) # Total value of portfolio after growth
        
    [eq_units_df, bd_units_df, bank_account] = trade(current_date, bank_account, eq_units_df, eq_price_df, bd_units_df, bd_price_df)

    previous_date = current_date
    prev_mkt_value = sum(eq_units_df[current_date]*eq_price_df[current_date])+ sum(bd_price_df[current_date] * bd_units_df[current_date])

    proj_period+=1

C:\Users\grego\Desktop\Open_Source_Economic_Model-main\BondClasses.py:533: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  notional_df.loc[asset_id],settings.modelling_date, proj_period,curves,bond_zspread_df.loc[asset_id][0])
C:\Users\grego\Desktop\Open_Source_Economic_Model-main\BondClasses.py:534: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bond_price_df.loc[asset_id][date_of_interest] = price
C:\Users\grego\Desktop\Open_Source_Economic_Model-main\BondClasses.py:533: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consis

In [232]:
print("Done")

Done
